# California Fire Information

In [1]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# set url
base_url = 'https://www.fire.ca.gov/incidents/'
res = requests.get(base_url)
# print status code
res.status_code

200

In [3]:
# create soup from base url
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
# examine soup object
# print(soup.prettify())

In [5]:
# create table 
table = soup.find('div',{'class':'featured-incidents'})

In [6]:
# examine table
# print(table.prettify())

In [7]:
# cell used to isolate info
for i in table.find_all('div',{'class':'featured-incident'}):
    print(i.find('div',{'class':'incident-description'}).find('strong').text)


9:30am
10/23/2019
7:13am
2:11pm
7:46am
9:20am
7:41am
3:06pm
10/24/2019
10/24/2019
10/22/2019
10/21/2019
10/14/2019
10/04/2019


In [8]:
# loop through list to obtain desired info and create dataframe
fires = []

for i in table.find_all('div',{'class':'featured-incident'}):
    fire = {}
    
    fire['name'] = i.find('div',{'class':'incident-label'}).text
    fire['lat'] = i.attrs['data-lat']
    fire['lng'] = i.attrs['data-long']
    fire['percent_contained'] = i.find('div',{'class':'incident-containment'}).attrs['style'].replace('width:', '').replace(';', '')
    fire['acres'] = i.find('span',{'class':'incident-acres'}).text.replace('acres', '')
    fire['current_as_of'] = i.find('div',{'class':'incident-description'}).find('strong').text
    fires.append(fire)

df = pd.DataFrame(fires)
df    

,acres,current_as_of,lat,lng,name,percent_contained
0,21900,9:30am,38.792458,-122.780053,Kincade Fire,5%
1,8799,10/23/2019,34.32988,-118.48161,Saddle Ridge Fire,97%
2,4300,7:13am,34.472778,-118.368056,Tick Fire,5%
3,97,2:11pm,33.055124,-116.742508,Sawday Fire,15%
4,95,7:46am,37.166674,-122.361112,Cabrillo Fire,55%
5,58,9:20am,37.896231,-122.636471,Muir Fire,50%
6,50,7:41am,37.62782,-121.68598,Mines Fire,60%
7,10,3:06pm,33.259861,-117.027333,Miller Fire,%
8,Not a CAL FIRE Incident,10/24/2019,34.183624,-117.274276,Old Water Fire,%
9,Not a CAL FIRE Incident,10/24/2019,33.351145,-117.403719,Oak Fire,%


In [9]:
df.dtypes

acres                object
current_as_of        object
lat                  object
lng                  object
name                 object
percent_contained    object
dtype: object

In [10]:
# cast lat and lng to float
df[['lat', 'lng']] = df[['lat', 'lng']].astype(float)
df.dtypes

acres                 object
current_as_of         object
lat                  float64
lng                  float64
name                  object
percent_contained     object
dtype: object

## Display fires on map

In [11]:
import folium

In [12]:
# California latitude and longitude values
latitude = 36.7783
longitude = -119.4179

# create map and display it
cali_map = folium.Map(location=[latitude, longitude], zoom_start=6, tiles='Stamen Terrain')

# add markers for fires
labels = list(df.name)

latitudes = list(df.lat)
longitudes = list(df.lng)


for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng],popup=label).add_to(cali_map) 

# display the map of California
cali_map